In [1]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import re
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
stopwords = set(stopwords.words('english'))  # Ensure you've downloaded the stopwords set from nltk.

def preprocess(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+','', text)  # URL removal
    text = re.sub('<.*?>+', '', text)  # HTML tags removal
    text = re.sub('\[.*?\]', '', text)  # Square brackets removal
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Punctuation removal
    text = re.sub('\n', '', text)  # Newline removal
    text = re.sub('\w*\d\w*', '', text)  # Words with digits removal
    text = ' '.join([word for word in text.split(' ') if word not in stopwords])  # Stopwords removal
    text = ' '.join([stemmer.stem(word) for word in text.split(' ')])  # Stemming
    return text


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

In [9]:
df1 = pd.read_csv('/content/drive/MyDrive/sentiment_analysis_major_project/IMDB Dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/sentiment_analysis_major_project/twitter_data.csv')

In [10]:
df1.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
df1.rename({'review':'text'},axis=1,inplace=True)
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder()
df1['sentiment'] = lenc.fit_transform(df1['sentiment'])

In [12]:
df1.head()

,text,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
df2.head()

,Unnamed: 0,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797.0,27400.0,105.0
2,2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265.0,470.0,164.0
4,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [14]:
df2 = df2[['text','sentiment']]

In [15]:
df2.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [16]:
df2 = df2[df2['sentiment']!='neutral']

In [17]:
df2.head()

,text,sentiment
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
6,2am feedings for the baby are fun when he is a...,positive


In [18]:
df2['sentiment'] = lenc.fit_transform(df2['sentiment'])

In [19]:
df2.head()

,text,sentiment
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0
6,2am feedings for the baby are fun when he is a...,1


In [20]:
df = pd.concat([df1,df2],ignore_index=True)

In [21]:
df.head()

,text,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [22]:
df.shape

(69748, 2)

In [23]:
df.isna().sum()

text         1281
sentiment       0
dtype: int64

In [24]:
df.duplicated().sum()

1698

In [25]:
df.dropna(inplace=True)

In [26]:
df.drop_duplicates(inplace=True)

In [27]:
df.isna().sum()

text         0
sentiment    0
dtype: int64

In [28]:
df.duplicated().sum()

0

In [29]:
df.shape

(68049, 2)

In [30]:
df.sentiment.unique()

array([1, 0])

In [31]:
df.sentiment.value_counts()

1    34569
0    33480
Name: sentiment, dtype: int64

In [32]:
df['text'] = df['text'].apply(preprocess)

In [33]:
df.head()

,text,sentiment
0,one review mention watch oz episod youll hook...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic there famili littl boy jake think there ...,0
4,petter mattei love time money visual stun film...,1


In [34]:
df.to_csv('preprocessed_data.csv')